# OpenAI Document Search with Langchain

This example shows how to use the Python [langchain](https://python.langchain.com/docs/get_started/introduction) library to run a text-generation request against [OpenAI's](https://platform.openai.com/) API, then augment that request using the text stored in a collection of local PDF documents.

**Requirements:**
- You will need an access key to OpenAI's API key, which you can sign up for at (https://dashboard.cohere.com/welcome/login). A free trial account will suffice, but will be limited to a small number of requests.
- After obtaining this key, store it in plain text in your home in directory in the `~/.openai.key` file.
- (Optional) Upload some pdf files into the `source_documents` subfolder under this notebook. We have already provided some sample pdfs, but feel free to replace these with your own.

## Set up the RAG workflow environment

In [1]:
from getpass import getpass
import os
from pathlib import Path

from langchain.chains import RetrievalQA
from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.schema import HumanMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

Set up some helper functions:

In [2]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

Make sure other necessary items are in place:

In [3]:
try:
    os.environ["OPENAI_API_KEY"] = open(Path.home() / ".openai.key", "r").read().strip()
except Exception as err:
    print(f"Could not read your OpenAI key. Please make sure this is available in plain text under your home directory in ~/.openai.key: {err}")

# Look for the source_documents folder and make sure there is at least 1 pdf file here
contains_pdf = False
directory_path = "./source_documents"
if not os.path.exists(directory_path):
    print(f"ERROR: The {directory_path} subfolder must exist under this notebook")
for filename in os.listdir(directory_path):
    contains_pdf = True if ".pdf" in filename else contains_pdf
if not contains_pdf:
    print(f"ERROR: The {directory_path} subfolder must contain at least one .pdf file")

## Start with a basic generation request without RAG augmentation

Let's start by asking OpenAI a difficult, domain-specific question we don't expect it to have an answer to. A simple question like "*What is the capital of France?*" is not a good question here, because that's basic knowledge that we expect the LLM to know.

Instead, we want to ask it a question that is very domain-specific that it won't know the answer to. A good example would an obscure detail buried deep within a company's annual report. For example:

"*How many Vector scholarships in AI were awarded in 2022?*"

In [4]:
query = "How many Vector scholarships in AI were awarded in 2022?"

## Now send the query to OpenAI

In [5]:
llm = ChatOpenAI()
message = [
    HumanMessage(
        content=query
    )
]
result = llm(message)
print(f"Result: \n\n{result.content}")

/ssd003/projects/aieng/public/rag_bootcamp/envs/rag_dataloaders/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Result: 

There is no specific information available regarding the number of Vector scholarships in AI awarded in 2022. It is recommended to check the official website of Vector Institute for any updates on the number of scholarships awarded in 2022.


Without additional information, Cohere is unable to answer the question correctly. **Vector in fact awarded 109 AI scholarships in 2022.** Fortunately, we do have that information available in Vector's 2021-22 Annual Report, which is available in the `source_documents` folder. Let's see how we can use RAG to augment our question with a document search and get the correct answer.

## Ingestion: Load and store the documents from source_documents

Start by reading in all the PDF files from `source_documents`, break them up into smaller digestible chunks, then encode them as vector embeddings.

In [6]:
# Load the pdfs
loader = PyPDFDirectoryLoader(directory_path)
docs = loader.load()
print(f"Number of source documents: {len(docs)}")

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_documents(docs)
print(f"Number of text chunks: {len(chunks)}")

# Define the embeddings model
model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

print(f"Setting up the embeddings model...")
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

print(f"Done")

Number of source documents: 147
Number of text chunks: 800
Setting up the embeddings model...
Done


# Retrieval: Make the document chunks available via a retriever

The retriever will identify the document chunks that most closely match our original query. (This takes about 1-2 minutes)

In [7]:
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

# Retrieve the most relevant context from the vector store based on the query(No Reranking Applied)
docs = retriever.get_relevant_documents(query)

Let's see what results it found. Important to note, these results are in the order the retriever thought were the best matches.

In [8]:
pretty_print_docs(docs)

Document 1:

26 
  VECTOR SCHOLARSHIPS IN 
AI ATTRACT TOP TALENT TO ONTARIO UNIVERSITIES 
109 
Vector Scholarships in AI awarded 
34 
Programs 
13 
Universities 
351 
Scholarships awarded since the 
program launched in 2018 Supported with funding from the Province of Ontario, the Vector Institute Scholarship in Artifcial Intelligence (VSAI) helps Ontario universities to attract the best and brightest students to study in AI-related master’s programs. 
Scholarship recipients connect directly with leading
----------------------------------------------------------------------------------------------------
Document 2:

Vector Institute   |   Annual Report 2022-2023      26
VECTOR SCHOLARSHIPS IN AI DRAW EXCEPTIONAL TALENT TO ONTARIO UNIVERSITIES
Thanks to funding generously provided by the Province of Ontario, 
Vector Scholarships in AI (VSAI) enable the recruitment of talented and promising students from around the world into AI-related master's programs at universities across Ontario. 
R

These results seem to somewhat match our original query, but we still can't seem to find the information we're looking for. Let's try sending our LLM query again including these results, and see what it comes up with.

In [9]:
print(f"Sending the RAG generation with query: {query}")
qa = RetrievalQA.from_chain_type(llm=llm,
        chain_type="stuff",
        retriever=retriever)
print(f"Result:\n\n{qa.run(query=query)}") 

Sending the RAG generation with query: How many Vector scholarships in AI were awarded in 2022?


/ssd003/projects/aieng/public/rag_bootcamp/envs/rag_dataloaders/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Result:

In 2022, a total of 351 Vector Scholarships in AI were awarded.


# Reranking: Improve the ordering of the document chunks

In [10]:
embeddings = OpenAIEmbeddings()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)
compressed_docs = compression_retriever.get_relevant_documents(query)

Now let's see what the reranked results look like:

In [11]:
pretty_print_docs(compressed_docs)

Document 1:

26 
  VECTOR SCHOLARSHIPS IN 
AI ATTRACT TOP TALENT TO ONTARIO UNIVERSITIES 
109 
Vector Scholarships in AI awarded 
34 
Programs 
13 
Universities 
351 
Scholarships awarded since the 
program launched in 2018 Supported with funding from the Province of Ontario, the Vector Institute Scholarship in Artifcial Intelligence (VSAI) helps Ontario universities to attract the best and brightest students to study in AI-related master’s programs. 
Scholarship recipients connect directly with leading
----------------------------------------------------------------------------------------------------
Document 2:

Vector Institute   |   Annual Report 2022-2023      26
VECTOR SCHOLARSHIPS IN AI DRAW EXCEPTIONAL TALENT TO ONTARIO UNIVERSITIES
Thanks to funding generously provided by the Province of Ontario, 
Vector Scholarships in AI (VSAI) enable the recruitment of talented and promising students from around the world into AI-related master's programs at universities across Ontario. 
R

Lastly, let's run our LLM query a final time with the reranked results:

In [12]:
qa = RetrievalQA.from_chain_type(llm=llm,
        chain_type="stuff",
        retriever=compression_retriever)

print(f"Result:\n\n {qa.run(query=query)}")

Result:

 The number of Vector scholarships in AI awarded in 2022 is not explicitly mentioned in the provided context.
